## Download dataset

In [1]:
from google.colab import userdata
pat = userdata.get('SECRET_gitPAT')

!if [ ! -d "econ2355-scrape" ]; then git clone https://{pat}@github.com/tanxpyox/econ2355-scrape.git; else cd econ2355-scrape; git fetch && git pull origin; cd ..; fi

Cloning into 'econ2355-scrape'...
remote: Enumerating objects: 3203, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 3203 (delta 29), reused 88 (delta 11), pack-reused 3092
Receiving objects: 100% (3203/3203), 26.45 MiB | 19.23 MiB/s, done.
Resolving deltas: 100% (905/905), done.
Updating files: 100% (2018/2018), done.


## Train Model

In [2]:
# !pip install pytorch-metric-learning
# from pytorch_metric_learning import losses
import torch
from tqdm import tqdm
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # Set device

In [3]:
# Create dataset object

import pandas as pd

DV_name = 'bin_labels'

# TODO: Finish implementing the TweetDataset object
class EscalationDataset(torch.utils.data.Dataset):
  def __init__(self, df):
    #df = df.loc[df['labels'] != 2]
    self.X = df['answer'].tolist()
    self.y = df[DV_name].tolist()

  def __len__(self):
    return len(self.X)

  def __getitem__(self, i):
    return self.X[i], self.y[i]

raw_dataset = pd.read_csv("econ2355-scrape/full_dataset.csv")
dataset = EscalationDataset(raw_dataset)
n_labels = raw_dataset[DV_name].value_counts().size

In [4]:
# Download model: ZH bert
from transformers import pipeline, BertTokenizer, BertModel, BertForSequenceClassification, BertForPreTraining
# import faiss
# from pytorch_metric_learning.samplers import MPerClassSampler
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
base_model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels = 2, output_attentions=True)

# Attach classification head to BERT model

# class Classifier(torch.nn.Module):
#   def __init__(self, bert_model):
#     super(Classifier, self).__init__()
#     self.bert_model = bert_model
#     self.dropout = torch.nn.Dropout(p = 0.3)
#     self.linear = torch.nn.Linear(768, n_labels)
#     self.softmax = torch.nn.Softmax(dim = 1)

#   def forward(self, input_ids, mask):
#     _, pooled_output = self.bert_model(input_ids = input_ids, attention_mask = mask, return_dict = False)
#     return self.softmax(self.linear(self.dropout(pooled_output)))
#
# model = Classifier(base_model)

model = base_model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# TODO: Split data into three sets, with proportions defined above
train_set, val_set, test_set = torch.utils.data.random_split(dataset, [0.8, 0.1, 0.1])

# TODO: Create dataloaders for the data
train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=4, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset=val_set, batch_size=4, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=1, shuffle=True)

In [6]:
# Prepare for training
learning_rate = 1e-6

#TODO: Move the model to the GPU
model = model.to(device)

# TODO: create an Adam optimizer for the model
optim = torch.optim.Adam(model.parameters(), lr = learning_rate)

# TODO: create a cross entropy loss function
loss_fn = torch.nn.CrossEntropyLoss()

In [ ]:
num_epochs = 5

for _ in range(num_epochs):

  model.train()
  for X, y in tqdm(train_loader):
    # TODO: tokenize the input
    tokens = tokenizer(X, return_tensors='pt', add_special_tokens = True,
                       max_length = 512,
                       padding=True, truncation = True)

    # TODO: move the encoded input ids, encoded attention mask, and y to the GPU
    ids = tokens.input_ids.to(device)
    attn = tokens.attention_mask.to(device)
    y = y.to(device)

    # TODO: Run the model over the input, compute the loss, zero the gradients, backpropagate, and step the optimzer
    preds = model(ids, attn).logits
    loss = loss_fn(preds, y)
    optim.zero_grad()
    loss.backward()
    optim.step()

  model.eval()
  n_correct = 0
  for X, y in tqdm(val_loader):
    # TODO: complete the evaulation loop, computing validation accuracy
    tokens = tokenizer(X, return_tensors='pt', add_special_tokens = True,
                       max_length = 512,
                       padding=True, truncation = True)
    ids = tokens.input_ids.to(device)
    attn = tokens.attention_mask.to(device)

    preds = model(ids, attn).logits
    n_correct += sum(torch.eq(torch.argmax(preds.cpu(), 1), y))

  print(f'Val Accuracy epoch {_ + 1}: {n_correct / len(val_set)}')

In [8]:
model.eval()
n_correct = 0
for X, y in tqdm(test_loader):
  # TODO: complete the evaulation loop, computing validation accuracy
  tokens = tokenizer(X, return_tensors='pt', add_special_tokens = True,
                      max_length = 512,
                      padding=True, truncation = True)
  ids = tokens.input_ids.to(device)
  attn = tokens.attention_mask.to(device)

  preds = model(ids, attn).logits
  n_correct += sum(torch.eq(torch.argmax(preds.cpu(), 1), y))

print(f'Eval Accuracy: {n_correct / len(test_set)}')

100%|██████████| 75/75 [00:02<00:00, 37.03it/s]

Eval Accuracy: 0.8133333325386047


In [9]:
# Visualisation test
!pip install transformers-interpret
from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)

test_text = """
汪文斌：美国向台湾地区出售武器严重违反一个中国原则和中美三个联合公报特别是“八·一七”公报规定，干涉中国内政，严重损害中国主权和安全利益，向“台独”分裂势力发出错误信号，严重损害中美关系和台海和平稳定。中方对此坚决反对。
中方敦促美方恪守一个中国原则和中美三个联合公报规定，停止售台武器和美台军事联系，撤销有关对台军售计划，以免对中美关系和台海和平稳定造成进一步损害。中方将采取正当、必要措施，坚定捍卫国家主权和安全利益。
"""

cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)

word_attributions = cls_explainer(test_text)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x

In [10]:
cls_explainer.visualize()

In [11]:
test_text = """
毛宁：习近平主席在美国会见日本首相岸田文雄时，就日本福岛核污染水排海问题交换了意见。习近平主席指出，日本福岛核污染水排海事关人类健康、全球海洋环境和国际公共利益，日方应当严肃对待国内外的合理关切，本着负责任和建设性的态度妥善处理。双方同意本着建设性的态度，通过磋商谈判，找到解决福岛核污染水排海问题的合适途径。
随着越来越多的福岛核污染水排入大海，国际社会迫切要求建立长期有效的国际监测安排，日方应当以严肃认真的态度回应国际社会关切，以负责任方式处置核污染水，全面配合建立有日本周边邻国等利益攸关方实质性参与、长期有效的国际监测安排，防止核污染水排海造成不可挽回的后果。"""

word_attributions = cls_explainer(test_text)
cls_explainer.visualize()
